In [85]:
from pathlib import Path

import numpy as np
from numba import njit, prange

import pandas as pd

In [86]:
df = pd.read_csv(Path('..', '..', '..', 'data', 'iris_csv.csv'))

for c in df.columns[0:4]:
    df[c] = (df[c]-df[c].mean())/df[c].std()

df['synth1'] = df['petallength']*df['petalwidth']
df['synth2'] = df['sepallength']*df['petallength']
df['synth3'] = df['sepallength']*df['petalwidth']

for name in df['class'].unique():
    df[f'label-{name}'] = df['class'].map(lambda x: 1 if x == name else 0)

In [87]:
np.random.seed(0)

setosa_idxs = np.arange(0, 50)
versicolor_idxs = np.arange(50, 100)
virginica_idxs = np.arange(100, 150)

p = np.random.permutation(np.arange(50))

setosa_train_idxs = setosa_idxs[p[0:10]]
setosa_test_idxs = setosa_idxs[p[10:]]

versicolor_train_idxs = versicolor_idxs[p[0:10]]
versicolor_test_idxs = versicolor_idxs[p[10:]]

virginica_train_idxs = virginica_idxs[p[0:10]]
virginica_test_idxs = virginica_idxs[p[10:]]

feature_columns = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']
label_columns = ['label-Iris-setosa', 'label-Iris-versicolor', 'label-Iris-virginica']

xTrain = np.vstack([
    df.iloc[setosa_train_idxs][feature_columns],
    df.iloc[versicolor_train_idxs][feature_columns],
    df.iloc[virginica_train_idxs][feature_columns]
])

yTrain = np.vstack([
    df.iloc[setosa_train_idxs][label_columns],
    df.iloc[versicolor_train_idxs][label_columns],
    df.iloc[virginica_train_idxs][label_columns]
])

xTest = np.vstack([
    df.iloc[setosa_test_idxs][feature_columns],
    df.iloc[versicolor_test_idxs][feature_columns],
    df.iloc[virginica_test_idxs][feature_columns]
])

yTest = np.vstack([
    df.iloc[setosa_test_idxs][label_columns],
    df.iloc[versicolor_test_idxs][label_columns],
    df.iloc[virginica_test_idxs][label_columns]
])

In [88]:
@njit(fastmath=True)
def F(x: np.ndarray) -> np.ndarray:
    # return x
    return np.clip(x,-1, 1)
    # return np.tanh(x)


@njit(fastmath=True)
def dF(x: np.ndarray) -> np.ndarray:
    # return np.ones(x.shape)
    return np.array([0 if xi <= -1 or xi >= 1 else 1 for xi in x])
    # return 1-np.square(np.tanh(x))


@njit(fastmath=True)
def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(np.zeros(x.shape), x)


@njit(fastmath=True)
def dRelu(x: np.ndarray) -> np.ndarray:
    return 1 * (x > 0)


@njit(fastmath=True)
def softmax(x: np.ndarray) -> float:
    y = np.exp(x)
    return y/np.sum(y)

In [89]:
@njit(fastmath=True)
def grads(xBatch: np.ndarray, yBatch: np.ndarray, Wh: np.ndarray, w: np.ndarray, Bh: np.ndarray, b: np.ndarray) -> tuple[np.ndarray]:
    n = xBatch.shape[0]

    dWh = np.zeros(Wh.shape)
    dw = np.zeros(w.shape)
    
    dBh = np.zeros(Bh.shape)
    db = np.zeros(b.shape)
    
    for i in prange(n):
        Uh = xBatch[i] @ Wh + Bh
        h = relu(Uh)
        y = softmax(h @ w + b)

        dw += (y - yBatch[i]) * np.atleast_2d(h).T
        db += y - yBatch[i]
        
        dWh += (y - yBatch[i]) @ w.T * dRelu(Uh) * np.atleast_2d(xBatch[i]).T
        dBh += (y - yBatch[i]) @ w.T * dRelu(Uh)
    
    return (dWh, dw, dBh, db)

class Perceptron:
    def __init__(self, nIn: int, nHidden: int, nOut: int) -> None:
        self.nIn = nIn
        self.nHidden = nHidden
        self.nOut = nOut
        
        self.Wh: np.ndarray = np.random.uniform(-1, 1, (nIn, nHidden))
        self.w: np.ndarray = np.random.uniform(-1, 1, (nHidden, nOut))
        
        self.Bh: np.ndarray = np.zeros((nHidden))
        self.b: np.ndarray = np.zeros((nOut))

    def predict(self, x:np.ndarray) -> np.ndarray:
        h = relu(x @ self.Wh + self.Bh)
        return softmax(h @ self.w + self.b)

    def train(self, xTrain: np.ndarray, yTrain: np.ndarray, lr, batch_size, max_iter) -> None:
        n = xTrain.shape[0]

        for i in range(max_iter):
            idxs = np.random.choice(a=np.arange(n), size=batch_size, replace=False)

            dWh, dw, dBh, db = grads(xTrain[idxs], yTrain[idxs], self.Wh, self.w, self.Bh, self.b)

            self.Wh -= lr*dWh
            self.w -= lr*dw

            self.Bh -= lr*dBh
            self.b -= lr*db

    def loss(self, x: np.ndarray, y: np.ndarray) -> float:        
        Ypred = np.array([self.predict(xi) for xi in x])
        h = -np.sum(y*np.log(Ypred), axis=1)
        return 1/x.shape[0] * np.sum(h)

In [90]:
nIn = 4
nHidden = 12
nOut = 3

learning_rate = 1e-2
batch_size = 30
max_iter = 1000

model = Perceptron(nIn, nHidden, nOut)

print('untrained loss: ', model.loss(xTest, yTest).round(3))

model.train(
    xTrain,
    yTrain,
    learning_rate,
    batch_size,
    max_iter
)

print('trained loss: ', model.loss(xTest, yTest).round(3))

TP_count = 0
for x, y in zip(xTest, yTest):
    yPred = model.predict(x)
    TP_count += 1 if np.argmax(y) == np.argmax(yPred) else 0

accuracy = TP_count / xTest.shape[0]
print('accuracy: ', accuracy)

untrained loss:  1.54
trained loss:  0.266
accuracy:  0.95


In [91]:
for x,y in zip(xTest, yTest):
    yPred = model.predict(x)
    print('X' if np.argmax(y) != np.argmax(yPred) else ' ', y, yPred.round(2))

  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [0 1 0] [0.   0.69 0.31]
  [0 1 0] [0.   0.64 0.35]
  [0 1 0] [0. 1. 0.]
X [0 1 0] [0.01 0.45 0.54]
  [0 1 0] [0. 1. 0.]
  [0 1 0] [0.14 0.86 0.  ]
  [0 1 0] 